In [56]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
from io import BytesIO
import re
import time
from openpyxl import load_workbook

# ------ STREAMLIT APP ------ #

st.title("Excel Form Processor with Categories & Repeater Mapping")

# Step 1: Upload Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

if uploaded_file:
    xls = pd.ExcelFile(uploaded_file)

    # Must contain formFields
    if "formFields" not in xls.sheet_names:
        st.error("The Excel file must contain a sheet named 'formFields'.")
    else:
        st.success("Found 'formFields' sheet. Processing automatically...")

        df = pd.read_excel(uploaded_file, sheet_name="formFields")

        # Fixed required columns
        field_name_col = "name"
        field_type_col = "input_type"
        field_order_col = "order"
        form_col = "formName"

        # Fixed category columns
        fixed_category_cols = [
            "is_editable", "is_hidden", "is_required", "is_nullable",
            "is_multiselect", "is_richtext", "editor_height",
            "is_encrypted", "is_conditional"
        ]
        selected_category_cols = [c for c in fixed_category_cols if c in df.columns]

        # Load fieldMapping sheet
        field_mapping_df = pd.DataFrame(columns=["formName", "fieldName"])
        if "fieldMapping" in xls.sheet_names:
            field_mapping_df = pd.read_excel(uploaded_file, sheet_name="fieldMapping")

        # Process all unique forms
        all_forms = sorted(df[form_col].dropna().unique())
        st.write(f"Processing {len(all_forms)} forms automatically...")

        output = BytesIO()
        progress_bar = st.progress(0)

        uploaded_file.seek(0)
        wb = load_workbook(uploaded_file)

        for i, full_form_name in enumerate(all_forms):

            # --------------------- #
            #  SHEET NAME EXTRACTION
            # --------------------- #

            # Example:
            # BTRC Approval_New Sales_Assign_QC_Agent_691c51e5a6fc3
            parts = full_form_name.split("_")

            # Remove prefix: first two sections (BTRC Approval) + (New Sales)
            if len(parts) > 3:
                middle_parts = parts[2:-1]      # Take everything except prefix (2) and suffix (1)
                short_name = "_".join(middle_parts)
            else:
                short_name = full_form_name  # fallback if format unexpected

            # Replace invalid Excel chars
            safe_short_name = re.sub(r'[\\/*?:\[\]]', '_', short_name)

            # Trim to Excel 31 char limit
            safe_short_name = safe_short_name[:31]

            # Filter rows for the form
            form_df = df[df[form_col] == full_form_name].sort_values(by=field_order_col)

            final_df = form_df[[field_name_col, field_type_col, field_order_col]].copy()

            # Add "Repeater" column
            repeater_list = []
            filtered_mapping = field_mapping_df[field_mapping_df['formName'] == full_form_name]
            mapped_fields = set(filtered_mapping['fieldName'].dropna())

            for field in final_df[field_name_col]:
                repeater_list.append("Yes" if field in mapped_fields else "")

            final_df.insert(len(final_df.columns), "Repeater", repeater_list)

            # Add Categories column
            if selected_category_cols:
                categories_list = []
                for idx, row in form_df.iterrows():
                    selected = [col for col in selected_category_cols if row.get(col) == 1]
                    selected.sort()
                    categories_list.append(",".join(selected) if selected else "")
                final_df['Categories'] = categories_list

            # Remove existing sheet if it already exists
            if safe_short_name in wb.sheetnames:
                wb.remove(wb[safe_short_name])

            # Create new sheet
            ws = wb.create_sheet(title=safe_short_name)

            # Write header row:
            ws.cell(row=1, column=1, value="Form")               # A1
            ws.cell(row=1, column=2, value=full_form_name)       # B1

            # Write dataframe headers starting from row 2
            for c_idx, header in enumerate(final_df.columns, start=1):
                ws.cell(row=2, column=c_idx, value=header)

            # Write DF contents starting from row 3
            for r_idx, row in enumerate(final_df.values, start=3):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Update progress
            progress = int((i + 1) / len(all_forms) * 100)
            progress_bar.progress(progress)
            time.sleep(0.05)

        # Save final output
        wb.save(output)

        st.success("Modified Excel created successfully!")
        st.download_button(
            label="Download Modified Excel",
            data=output.getvalue(),
            file_name="modified_original_with_categories_and_repeater.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

Overwriting app.py


In [57]:
# Install packages (if not installed already)
!pip install streamlit pyngrok openpyxl --quiet

from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Set your ngrok token
ngrok.set_auth_token("35gXlCpbnwCAIJ9I2xiA1GXDu2u_3UoQf6QeGYhFC9MxYapgo")

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Wait a few seconds for Streamlit to start
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://armless-twirly-alanna.ngrok-free.dev" -> "http://localhost:8501"
